# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''
import pandas as pd
reviews = pd.read_csv('IMDB Dataset.csv')
reviews=reviews.truncate(before = 0, after = 399)


reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
395,I commented on this when it first debuted and ...,negative
396,Just because someone is under the age of 10 do...,negative
397,"""Hey Babu Riba"" is a film about a young woman,...",positive
398,"After a snowstorm, the roads are blocked and t...",negative


In [2]:
#Split the data into train and split

from sklearn.model_selection import train_test_split
r = reviews['review']       
s = reviews['sentiment']

In [3]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0
x_train, x_test, y_train,  = train_test_split(r, s, test_size = 0.25)
y_train=y_train.replace('positive',1)
y_train=y_train.replace('negative',0)
y_test=y_test.replace('positive',1)
y_test=y_test.replace('positive',1)

train_reviews = pd.DataFrame(x_train, columns=[ 'review'])
test_reviews= pd.DataFrame(x_test, columns=[ 'review'])
train_sentiment=pd.DataFrame(y_train, columns=[ 'sentiment'])
test_sentiment=pd.DataFrame(y_test, columns=[ 'sentiment'])
test_sentiment

,sentiment
304,1
312,negative
228,1
76,1
241,1
...,...
123,negative
53,1
37,negative
144,negative


In [4]:
train_reviews.head()

,review
310,"Some people drift through life, moving from on..."
177,"Though I'd heard that ""Cama de Gato"" was the w..."
170,"I can't emphasize it enough, do *NOT* get this..."
180,It is so gratifying to see one great piece of ...
141,Ye Lou's film Purple Butterfly pits a secret o...


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [5]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize




        
        


train_reviews['Tokenized_Review'] = train_reviews['review'].str.lower().apply(word_tokenize)
train_reviews[['review','Tokenized_Review']][:10]

[nltk_data] Downloading package punkt to C:\Users\Dhruv
[nltk_data]     Gada\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,review,Tokenized_Review
310,"Some people drift through life, moving from on...","[some, people, drift, through, life, ,, moving..."
177,"Though I'd heard that ""Cama de Gato"" was the w...","[though, i, 'd, heard, that, ``, cama, de, gat..."
170,"I can't emphasize it enough, do *NOT* get this...","[i, ca, n't, emphasize, it, enough, ,, do, *, ..."
180,It is so gratifying to see one great piece of ...,"[it, is, so, gratifying, to, see, one, great, ..."
141,Ye Lou's film Purple Butterfly pits a secret o...,"[ye, lou, 's, film, purple, butterfly, pits, a..."
206,"If you liked William Hickey in ""Prizzi's Honor...","[if, you, liked, william, hickey, in, ``, priz..."
94,"I watched this series out of curiosity,wanting...","[i, watched, this, series, out, of, curiosity,..."
268,(When will I ever learn-?) The ecstatic review...,"[(, when, will, i, ever, learn-, ?, ), the, ec..."
308,"Well, there's no real plot to speak of, it's j...","[well, ,, there, 's, no, real, plot, to, speak..."
83,"""Fate"" leads Walter Sparrow to come in possess...","[``, fate, '', leads, walter, sparrow, to, com..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [6]:
'''Count tokens (words) in texts and add them to the lexicon'''

import pickle

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    # First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1
   
    

    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
    
    lexicon =  [token for token, count in token_counts.items() if count >= min_freq] 
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'some': 2, 'people': 3, 'drift': 4, 'through': 5, 'life': 6, ',': 7, 'moving': 8, 'from': 9, 'one': 10, 'thing': 11, 'or': 12, 'person': 13, 'to': 14, 'the': 15, 'next': 16, 'without': 17, 'batting': 18, 'an': 19, 'eye': 20, ';': 21, 'others': 22, 'latch': 23, 'on': 24, 'a': 25, 'cause': 26, 'another': 27, 'principle': 28, 'and': 29, 'remain': 30, 'adamant': 31, 'committed': 32, 'whatever': 33, 'it': 34, 'is': 35, '--': 36, 'figuratively': 37, 'literally': 38, 'they': 39, 'give': 40, 'their': 41, 'word': 42, 'stand': 43, 'by': 44, '.': 45, 'but': 46, 'we': 47, "'re": 48, 'all': 49, 'different': 50, '`': 51, 'made': 52, 'of': 53, 'clay': 54, "'": 55, 'as': 56, 'characters': 57, 'in': 58, 'this': 59, 'film': 60, 'puts': 61, 'which': 62, 'what': 63, 'makes': 64, 'so': 65, 'interesting': 66, 'are': 67, 'just': 68, 'plain': 69, 'crazy': 70, 'though': 71, 'maybe': 72, 'that': 73, "'s": 74, 'way': 75, 'you': 76, 'have': 77, 'be': 78, 'live': 79, 'among': 80, 'masses': 81, 'who': 82, 'knows':

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [7]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq]  for token_seq in token_seqs]
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]

,Tokenized_Review,Review_Idxs
310,"[some, people, drift, through, life, ,, moving...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 10, 13, 1..."
177,"[though, i, 'd, heard, that, ``, cama, de, gat...","[71, 438, 439, 440, 73, 441, 442, 443, 444, 44..."
170,"[i, ca, n't, emphasize, it, enough, ,, do, *, ...","[438, 465, 466, 776, 34, 342, 7, 579, 777, 157..."
180,"[it, is, so, gratifying, to, see, one, great, ...","[34, 35, 65, 854, 14, 266, 10, 855, 286, 53, 4..."
141,"[ye, lou, 's, film, purple, butterfly, pits, a...","[897, 898, 74, 60, 899, 900, 901, 25, 902, 903..."
206,"[if, you, liked, william, hickey, in, ``, priz...","[649, 76, 999, 1000, 1001, 58, 441, 1002, 74, ..."
94,"[i, watched, this, series, out, of, curiosity,...","[438, 450, 59, 1048, 155, 53, 1049, 7, 1050, 1..."
268,"[(, when, will, i, ever, learn-, ?, ), the, ec...","[135, 132, 825, 438, 581, 1104, 84, 138, 15, 1..."
308,"[well, ,, there, 's, no, real, plot, to, speak...","[250, 7, 186, 74, 130, 255, 247, 14, 1156, 53,..."
83,"[``, fate, '', leads, walter, sparrow, to, com...","[441, 1176, 445, 661, 1177, 1178, 14, 1179, 58..."


In [8]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    bow = numpy.array([numpy.bincount(numpy.array(idx_seq), minlength=matrix_length) 
                            for idx_seq in idx_seqs])
    return bow
    


bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[0 0 3 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 1 1 1]]
SHAPE: (300, 10537) 



,Word,Count
0,,0
1,<UNK>,0
2,some,3
3,people,4
4,drift,1
...,...,...
10532,indicating,0
10533,killer.,0
10534,disclosed,0
10535,four.,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    #find the biggest review's length and save it in the variable below
    max_seq_len =max([len(idx_seq) for idx_seq in idx_seqs])
    pad = pad_sequences(sequences=idx_seqs, maxlen=max_seq_len)
    return pad
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    
    


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[   2    3    4 ...   45  437   45]
 [   0    0    0 ...   29  775   45]
 [   0    0    0 ...   58   34   45]
 ...
 [   0    0    0 ...   74 6637   45]
 [   0    0    0 ...   45 9881 3341]
 [   0    0    0 ... 5308  445  138]]
SHAPE: (300, 1258) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [10]:
'''Create the model'''

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    input_layer = Input(shape=(None,))
    
    # Layer 2
    embedding_layer = Embedding(input_dim=n_input_nodes,output_dim=n_embedding_nodes,mask_zero=True)(input_layer)
    
    # Layer 3
    gru_layer = GRU(units=n_hidden_nodes)(embedding_layer)
    # Layer 4
    output_layer = Dense(units=1)(gru_layer)
    #Specify which layers are input and output, compile model with loss and optimization functions
    model = Model(inputs=[input_layer], outputs=output_layer)
    model.compile(loss="mean_squared_error", optimizer='adam')
    
    return model



In [11]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [12]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
tensor=tf.convert_to_tensor(train_sentiment['sentiment'])
rnn_model.fit(x=train_padded_idxs, y=tensor, batch_size=20, epochs=5)



Epoch 1/5
15/15 [==============================] - 139s 9s/step - loss: 0.3195
Epoch 2/5
15/15 [==============================] - 141s 9s/step - loss: 0.1996
Epoch 3/5
15/15 [==============================] - 148s 10s/step - loss: 0.0822
Epoch 4/5
15/15 [==============================] - 149s 10s/step - loss: 0.0429
Epoch 5/5
15/15 [==============================] - 149s 10s/step - loss: 0.0273


###  <font color='#6629b2'>Prediction</font>

In [15]:
'''Put test reviews in padded matrix just how we did for train_reviews'''
from nltk.tokenize import word_tokenize
test_reviews['Tokenized_Review'] = test_reviews['review'].str.lower().apply(word_tokenize)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'],lexicon=lexicon)
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])

print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

TEST INPUT:
 [[   0    0    0 ...    1    1   45]
 [   0    0    0 ...  445  631   45]
 [   0    0    0 ...  151  138   45]
 ...
 [   0    0    0 ...   39 1163 2942]
 [   0    0    0 ...   60 3185   45]
 [   0    0    0 ... 7537  935   45]]
SHAPE: (100, 1218) 



In [16]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [ ]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 
r2Score = r2_score(y_true=test_sentiment, y_pred=test_reviews['RNN_Pred_Rating'])
print('R2 score=',round(r2Score,6))

On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [22]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

EMBEDDING LAYER OUTPUT SHAPE: (100, 1218, 300)
[[-0.02921566 -0.02217088 -0.00992992 ... -0.0221552   0.04922057
   0.016933  ]
 [-0.02921566 -0.02217088 -0.00992992 ... -0.0221552   0.04922057
   0.016933  ]
 [-0.02921566 -0.02217088 -0.00992992 ... -0.0221552   0.04922057
   0.016933  ]
 ...
 [-0.02794824  0.00052643  0.03399228 ...  0.00226318 -0.02719034
  -0.0273417 ]
 [-0.02794824  0.00052643  0.03399228 ...  0.00226318 -0.02719034
  -0.0273417 ]
 [ 0.03592635 -0.03212464 -0.03186261 ... -0.0101424   0.01081498
  -0.029898  ]]
